In [1]:
from model_functions import analyze_token_sequence, predict, write_midi, get_token_flags
import json
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer

PATH_VOCAB = "../0_data/5_vocabs"
PATH_MODELS = "../0_data/7_models"
PATH_MODELS_CONFIG = "../0_data/7_models/config"
PATH_PRED = "../0_data/8_predictions"
PATH_TOKENS = "../0_data/8_predictions/tokens"
PATH_MIDI = "../0_data/8_predictions/midi"

for path in [PATH_PRED, PATH_TOKENS, PATH_MIDI]:
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
vocab_configs = {
    "a1" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "a2" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": True,
    },
    "a3" : {
        "pitch_range": 128,
        "duration_steps": 32,
        "triole_tokens": False,
    },
    "b" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "c" : {
        "pitch_range": 36,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "d" : {
        "pitch_range": 36,
        "duration_steps": 32,
        "triole_tokens": True,
    }
}

In [3]:
model_df = pd.read_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx", index_col="Unnamed: 0")
model_df

,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1,1024,256,4,6,0,0.001,50,4,yes,717.6662,11.96,1.373842,42,423.106567,NaN,NaN,NaN
1,a2,1024,256,4,6,0,0.001,50,4,yes,716.0189,11.93,1.140858,49,472.513916,NaN,NaN,NaN
2,a3,1024,256,4,6,0,0.001,50,4,yes,696.7114,11.61,1.342281,34,252.672455,NaN,NaN,NaN
3,b,1024,256,4,6,0,0.001,50,4,yes,636.9317,10.62,1.325169,27,306.182312,NaN,NaN,NaN
4,c,1024,256,4,6,0,0.001,50,4,yes,656.0490,10.93,1.362022,32,810.230347,NaN,NaN,NaN
5,d,1024,256,4,6,0,0.001,50,4,yes,598.8393,9.98,1.167735,50,486.180145,NaN,NaN,NaN


In [4]:
TICKS_PER_BEAT = 1024
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/32
    
for index, row in model_df.iterrows():

    # only models that ran
    if row["ran"] == "no" or pd.notnull(row["incorrect_notes"]) or row["runtime"] == "too big - cuda error":
        continue
    
    model_name = row["name"]
    if not os.path.exists(f"{PATH_MIDI}/{model_name}"):
        os.makedirs(f"{PATH_MIDI}/{model_name}")

    # get token flags and duration bins
    token_flags = get_token_flags(vocab_configs[model_name])
    duration_steps = vocab_configs[model_name]["duration_steps"]
    duration_bins = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*duration_steps)+1, TICKS_PER_MIN_DURATION, dtype=int)

    # create tokenizer
    tokenizer = GPT2Tokenizer(
        vocab_file=f"{PATH_VOCAB}/vocab_{model_name}.json", 
        merges_file=f"{PATH_VOCAB}/merges.txt")
    tokenizer.add_special_tokens({'pad_token': 'PAD', 'bos_token': 'BOS', 'eos_token': 'EOS',})
    
    # get vocabulary
    with open(f"{PATH_VOCAB}/vocab_{model_name}.json", "r") as fp:
        vocab = json.load(fp)
    token2word = {token: word for word, token in vocab.items()}

    # load model
    model = GPT2LMHeadModel.from_pretrained(f"{PATH_MODELS_CONFIG}/{model_name}/end_version")
    
    # make predictions save
    output = predict(model, tokenizer, samples=5, max_length=row["max_length"])
    data_generated = {"data": output}
    
    with open(f"{PATH_TOKENS}/{model_name}.json", "w") as fp:
        json.dump(data_generated, fp)
    
    # analyze tokens and save as midi_files
    correct_notes = 0
    incorrect_notes = 0
    for idx, pred in enumerate(output):
        an = analyze_token_sequence(pred, token_flags)
        correct_notes += an["start-pos-pitch-duration"] + an["start-pos-pitch-duration-dtriole"] + an["start-pos-ptriole-pitch-duration"] + an["start-pos-ptriole-pitch-duration-dtriole"]
        incorrect_notes += write_midi(output[idx], token2word, duration_bins, f"{PATH_MIDI}/{model_name}/generated_midi_{idx}.midi")
    
    model_df.at[index,"correct_notes"] = (correct_notes/5).__round__(2)
    model_df.at[index,"incorrect_notes"] = (incorrect_notes/5).__round__(2)
    model_df.at[index,"correct_rate"] = (correct_notes/(correct_notes+incorrect_notes)).__round__(2)
    
model_df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:211 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:214 for open-end generation.


midi saved in ../0_data/8_predictions/midi/a1/generated_midi_0.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/a1/generated_midi_1.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/a1/generated_midi_2.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/a1/generated_midi_3.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/a1/generated_midi_4.midi
Number of incorrect notes: 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:179 for open-end generation.


midi saved in ../0_data/8_predictions/midi/a2/generated_midi_0.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/a2/generated_midi_1.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/a2/generated_midi_2.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/a2/generated_midi_3.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/a2/generated_midi_4.midi
Number of incorrect notes: 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:211 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:119 for open-end generation.


midi saved in ../0_data/8_predictions/midi/b/generated_midi_0.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/b/generated_midi_1.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/b/generated_midi_2.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/b/generated_midi_3.midi
Number of incorrect notes: 4
midi saved in ../0_data/8_predictions/midi/b/generated_midi_4.midi
Number of incorrect notes: 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:90 for open-end generation.


midi saved in ../0_data/8_predictions/midi/c/generated_midi_0.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_1.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_2.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/c/generated_midi_3.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_4.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/d/generated_midi_0.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/d/generated_midi_1.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/d/generated_midi_2.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/d/generated_midi_3.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/d/generated_midi_4.midi
Number of incorrect notes: 0


,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1,1024,256,4,6,0,0.001,50,4,yes,717.6662,11.96,1.373842,42,423.106567,0.2,323.4,1.0
1,a2,1024,256,4,6,0,0.001,50,4,yes,716.0189,11.93,1.140858,49,472.513916,0.0,308.8,1.0
2,a3,1024,256,4,6,0,0.001,50,4,yes,696.7114,11.61,1.342281,34,252.672455,0.2,322.8,1.0
3,b,1024,256,4,6,0,0.001,50,4,yes,636.9317,10.62,1.325169,27,306.182312,1.2,324.4,1.0
4,c,1024,256,4,6,0,0.001,50,4,yes,656.0490,10.93,1.362022,32,810.230347,0.4,322.4,1.0
5,d,1024,256,4,6,0,0.001,50,4,yes,598.8393,9.98,1.167735,50,486.180145,0.2,253.0,1.0


In [5]:
model_df.to_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx")

run "tar chvfz predictions_midi.tar.gz *" in terminal midi folder to create and download zip

In [12]:
######## Andere Variante ########

In [13]:
outputs = model(inputs)
outputs.logits.shape

torch.Size([1, 1, 120])

In [14]:
outputs.logits

tensor([[[ 1.1162, -2.1350, -3.5313, -1.9780, -3.5414, -1.5776, -1.8704,
          -3.5441, -1.1535, -2.6672, -0.8764, -2.2812, -1.6335, -0.8708,
          -2.7905, -1.4720, -2.4843, -0.5825, -2.0775, -2.7744, -1.5724,
          -3.7235, -1.8095, -2.4802, -2.3428, -2.3748, -3.2765, -2.9800,
          -3.1671, -2.6769, -3.6342, -3.3965, -2.9433, -3.3021, -3.6102,
          -3.8760, -3.8078, -2.8941, -0.2965, -3.4592, -0.6409, -3.7882,
          -1.4499, -3.6240, -0.8940, -3.3144, -1.9948, -3.0505, -2.1582,
          -3.2950, -2.2917, -3.4952, -1.5516, -3.7166, -2.3756, -3.3413,
          -2.5331, -3.7758, -2.4212, -3.4614, -2.8825, -3.7335, -2.9825,
          -2.8619, -2.9172, -3.3198, -3.0439, -2.9400, -1.7772, -3.4107,
          -3.0528, -3.1332, -3.2200, -3.2291, -2.9769, -3.1939, -1.9817,
          -2.9937, -2.8389, -3.1996, -2.9903, -2.9885, -3.7754, -3.6936,
          -3.1062, -3.3149, -3.9010, -3.4983, -3.7158, -3.4427, -3.5639,
          -3.5325, -1.8639, -3.7709, -3.4870, -3.22

In [15]:
# Temperature value
temperature = 13

# Convert logits to probabilities using softmax with temperature
probs = F.softmax(outputs.logits / temperature, dim=-1)

# Sample a token from the probability distribution for each position in the sequence
predicted_tokens = torch.multinomial(probs.view(-1, probs.shape[-1]), num_samples=1).view(*probs.shape[:-1])
predicted_tokens

tensor([[24]], device='cuda:0')

In [16]:
tokenizer.decode(predicted_tokens[0], skip_special_tokens=False)

'Note-On_83'